In [1]:
import copy
import random
from collections import defaultdict
import pandas as pd
from src import DATA_DIR

In [4]:
class Resource:
    def __init__(self, name: str, capacity: int):
        self.name = name
        self.capacity = capacity
        self.available = capacity
        self.free_up_resources = defaultdict(int)



class Activity:
    def __init__(self, name: str, duration: float, resources: dict):
        self.name = name
        self.duration = duration
        self.resources = resources



class Variant:
    def __init__(self, name: str, activities: list):
        self.name = name
        self.activities = activities



class Trace:
    def __init__(self, variant, entry_time=0):
        self.variant_name = variant.name
        self.activities = variant.activities
        self.entry_time = entry_time
        self.todo_activities = copy.deepcopy(variant.activities)
        self.doing_activities = []  # {<activity>, <start_time>, <end_time>}
        self.done_activities = [] # {<activity>, <start_time>, <end_time>}
        self.start_time = None
        self.end_time = None




class Simulator:
    def __init__(self, traces, resources):
        self.current_time = 0
        self.traces = [Trace(variant, details['Entry_Time'])
                       for variant, details in traces
                       for ind in range(details['Count'])]
        self.resources = {resource.name: resource for resource in resources}



    def update_trace_doing_activities_state(self, trace):
        # Update the state of activities being done in the trace
        for ind, doing_item in enumerate(trace.doing_activities):
            if doing_item['end_time'] == self.current_time:
                done_activity = trace.doing_activities.pop(ind)
                trace.done_activities.append(done_activity)
                if len(trace.todo_activities) == 0 and len(trace.doing_activities) == 0:
                    trace.end_time = self.current_time



    def free_up_resources(self):
        # Free up resources that are no longer in use
        for resource_name, resource in self.resources.items():
            if self.current_time in resource.free_up_resources.keys():
                resource.available += resource.free_up_resources[self.current_time]
                resource.free_up_resources.pop(self.current_time)



    def allocate_resources(self, activity):
        # Check if required resources are available
        for resource_name, resource_required in activity.resources.items():
            resource = self.resources[resource_name]
            if resource.available < resource_required:
                return False
        # Allocate resources
        for resource_name, resource_required in activity.resources.items():
            self.resources[resource_name].available -= resource_required
            self.resources[resource_name].free_up_resources[self.current_time + activity.duration] += resource_required
        return True



    def run(self, duration):
        while self.current_time < duration:
            # Free up resources
            self.free_up_resources()

            for trace in self.traces:
                # Update trace state doing state
                self.update_trace_doing_activities_state(trace)

                # Pass done traces and the traces that their start data is bigger than current time                
                if len(trace.todo_activities) == 0 or len(trace.doing_activities) > 0 or trace.entry_time > self.current_time:
                    continue

                # Select first activity of trace
                activity = trace.todo_activities[0]

                # Allocate resources
                if not self.allocate_resources(activity):
                    continue

                # Assign Trace Start Time
                if len(trace.doing_activities) == 0 and len(trace.done_activities) == 0:
                    trace.start_time = self.current_time

                # Moving activity from todo to doing
                doing_activity = trace.todo_activities.pop(0)
                trace.doing_activities.append({'activity': doing_activity,
                                               'start_time': self.current_time,
                                               'end_time': self.current_time + activity.duration})

            self.current_time += 1



    def report_traces(self):
        trace_info_list = []
        for trace in self.traces:
            sum_of_activity = sum(item.duration for item in trace.activities)    
            duration = None
            waiting_time = None

            if trace.end_time:
                duration = trace.end_time - trace.start_time
                waiting_time = trace.end_time - trace.entry_time - sum_of_activity

            trace_info_list.append([trace.variant_name,
                                    trace.start_time,
                                    trace.end_time,
                                    sum_of_activity,
                                    duration,
                                    waiting_time])

        return pd.DataFrame(trace_info_list, columns=['Variant', 
                                                      'CaseStartTime',
                                                      'CaseEndTime',
                                                      'TotalDurationOfActivity',
                                                      'CaseTotalDuration',
                                                      'WaitingTime'])



    def create_eventlog(self):
        eventlog = []
        for case_ind, trace in enumerate(self.traces):
            for activity in trace.done_activities:
                eventlog.append([f"Case_{case_ind+1}",
                                 trace.variant_name,
                                 trace.start_time,
                                 trace.end_time,
                                 activity['activity'].name,
                                 activity['start_time'],
                                 activity['end_time']])

        return pd.DataFrame(eventlog, columns=['Case',
                                               'Variant', 
                                               'CaseStartTime',
                                               'CaseEndTime',
                                               'ActivityName',
                                               'ActivityStartTime',
                                               'ActivityEndTime'])

In [5]:
# Define resources
resource1 = Resource("Resource 1", 10)
resource2 = Resource("Resource 2", 10)
resource3 = Resource("Resource 3", 10)
resources = {resource1, resource2, resource3}


# Define activities
activity1 = Activity("Activity 1", 5, {"Resource 1": 1})
activity2 = Activity("Activity 2", 9, {"Resource 1": 1, "Resource 2": 2})
activity3 = Activity("Activity 3", 7, {"Resource 2": 1})
activity4 = Activity("Activity 4", 3, {"Resource 3": 1})


# Define Variants
variant1 = Variant("Variant 1", [activity1, activity2, activity3])
variant2 = Variant("Variant 2", [activity3, activity2, activity4])
variant3 = Variant("Variant 3", [activity4, activity1, activity3, activity2, activity1])


# Define Traces
traces = [(variant1, {'Count': 10, 'Entry_Time': 0}),
          (variant1, {'Count': 100, 'Entry_Time': 100}),
          (variant2, {'Count': 20, 'Entry_Time': 10}),
          (variant3, {'Count': 15, 'Entry_Time': 5})
         ]


# Create and run the simulator
simulator = Simulator(traces, resources)
simulator.run(duration=1000)

In [6]:
simulator.report_traces()

,Variant,CaseStartTime,CaseEndTime,TotalDurationOfActivity,CaseTotalDuration,WaitingTime
0,Variant 1,0,21,21,21,0
1,Variant 1,0,21,21,21,0
2,Variant 1,0,21,21,21,0
3,Variant 1,0,21,21,21,0
4,Variant 1,0,21,21,21,0
...,...,...,...,...,...,...
140,Variant 3,8,386,29,378,352
141,Variant 3,8,390,29,382,356
142,Variant 3,8,390,29,382,356
143,Variant 3,8,393,29,385,359


In [83]:
event_log = simulator.create_eventlog()
event_log

,Case,Variant,CaseStartTime,CaseEndTime,ActivityName,ActivityStartTime,ActivityEndTime
0,Case_1,Variant 1,0,23,Activity 1,0,5
1,Case_1,Variant 1,0,23,Activity 2,6,15
2,Case_1,Variant 1,0,23,Activity 3,16,23
3,Case_2,Variant 1,0,23,Activity 1,0,5
4,Case_2,Variant 1,0,23,Activity 2,6,15
...,...,...,...,...,...,...,...
460,Case_145,Variant 3,9,430,Activity 4,9,12
461,Case_145,Variant 3,9,430,Activity 1,20,25
462,Case_145,Variant 3,9,430,Activity 3,116,123
463,Case_145,Variant 3,9,430,Activity 2,416,425


In [17]:
class Student:
    def __init__(self, name, fname, national_code, age, gender, city):
        self.name = name
        self.fname = fname
        self.national_code = national_code
        self.age = age
        self.gender = gender
        self.city = city


    def calculate_grade(self):
        print('Salam')


    def say_hello():
        print('Salam')

In [18]:
student_1 = Student(name='Kourosh',
                    fname='Hasani',
                    national_code=615646561616,
                    age=25,
                    gender='male',
                    city='Tehran'
                    )

In [15]:
student_1.gender

'male'

In [16]:
student_1.calculate_grade()    ~     Student.calculate_grade(student_1)

Salam


In [20]:
Student.say_hello()

Salam


In [ ]:
student_1   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_2   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_3   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_4   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_5   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
...

In [52]:
class Student:
    def __init__(self, name, family_name, national_code, age):
        self.name = name
        self.family_name = family_name
        self.national_code = national_code
        self.age = age
        
        
    def say_hello(self):
        print('Hellooo')
        
        
    def func():
        pass

In [53]:
student = Student(name='Kourosh', family_name='Hasani', national_code="00000000000", age=30)

In [50]:
student.national_code

'00000000000'

In [54]:
student.say_hello() #     ~      Student.say_hello(student)

Hellooo


In [55]:
Student.func()

In [42]:
import pandas as pd

In [43]:
df = pd.DataFrame([])
df1 = pd.DataFrame([])
df2 = pd.DataFrame([])

In [47]:
dir(df)

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex_